In [1]:
import os
import pandas as pd

BASE_PATH = "E:/C-DAC/Major Project/AI-Based-Maritime-Port-Intelligence"

SILVER_PATH = os.path.join(BASE_PATH, "data", "silver")

ais = pd.read_csv(os.path.join(SILVER_PATH, "ais_cleaned.csv"))
container = pd.read_csv(os.path.join(SILVER_PATH, "container_tracking_cleaned.csv"))

C:\Users\harsh\AppData\Local\Temp\ipykernel_11784\65569494.py:8: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  ais = pd.read_csv(os.path.join(SILVER_PATH, "ais_cleaned.csv"))


In [2]:
ais.head()

,vessel_id,sog,cog,heading,length,width,data_source,event_time,lat,lon,vessel_type
0,219019621,0.0,86.0,86.0,9.0,4.0,kattegat,NaN,NaN,NaN,NaN
1,265628170,0.0,334.5,NaN,27.0,8.0,kattegat,NaN,NaN,NaN,NaN
2,219005719,0.0,208.7,NaN,11.0,4.0,kattegat,NaN,NaN,NaN,NaN
3,219028066,0.0,NaN,NaN,12.0,3.0,kattegat,NaN,NaN,NaN,NaN
4,212584000,0.0,153.0,106.0,99.0,13.0,kattegat,NaN,NaN,NaN,NaN


In [3]:
container.head()

,ship,imo,lat,lon,sog,cog,heading,depPort,arrPort,etaSchedule,eta,ata,etdSchedule,etd,atd
0,Megastar,9773064,60.1469,24.9135,6.3,219,216,FIHEL,EETLL,2018-05-04 21:30:00,2018-04-05 21:25:00,2018-04-05 21:23:00,2018-05-04 19:30:00,2018-07-04 15:29:00,2018-04-05 19:18:20
1,Megastar,9773064,60.1445,24.9100,11.6,217,217,FIHEL,EETLL,2018-05-04 21:30:00,2018-04-05 21:25:00,2018-04-05 21:29:00,2018-05-04 19:30:00,2018-07-04 15:29:00,2018-04-05 19:18:20
2,Megastar,9773064,60.1412,24.9061,14.2,198,199,FIHEL,EETLL,2018-05-04 21:30:00,2018-04-05 21:25:00,2018-04-05 21:30:00,2018-05-04 19:30:00,2018-07-04 15:29:00,2018-04-05 19:18:20
3,Star,9364722,59.4462,24.7726,3.7,17,159,EETLL,FIHEL,2018-05-04 21:30:00,2018-04-05 21:26:00,2018-04-05 21:46:00,2018-05-04 19:30:00,2018-07-04 15:25:00,2018-04-05 19:21:17
4,Megastar,9773064,60.1344,24.9056,15.9,179,179,FIHEL,EETLL,2018-05-04 21:30:00,2018-04-05 21:25:00,2018-04-05 21:32:00,2018-05-04 19:30:00,2018-07-04 15:29:00,2018-04-05 19:18:20


In [4]:
ais["event_time"] = pd.to_datetime(ais["event_time"], errors="coerce")

time_cols = ["etaSchedule", "eta", "ata", "etdSchedule", "etd", "atd"]
for col in time_cols:
    if col in container.columns:
        container[col] = pd.to_datetime(container[col], errors="coerce")


## Arrival & Departure Delay Columns

### Arrival Delay

In [5]:
container['arrival_delay_hrs'] = (container['ata'] - container['etaSchedule']).dt.total_seconds() / 3600

### Departure Delay

In [6]:
container['departure_delay_hrs'] = (container['atd'] - container['etdSchedule']).dt.total_seconds() / 3600

## Basic Waiting / Port Stay Time

In [7]:
container['port_stay_hrs'] = (container['atd'] - container['ata']).dt.total_seconds() / 3600

## Create Core Port Events Table

In [8]:
port_events = container[
    [
        "ship", "imo",
        "depPort", "arrPort",
        "etaSchedule", "eta", "ata",
        "etdSchedule", "etd", "atd",
        "arrival_delay_hrs",
        "departure_delay_hrs",
        "port_stay_hrs"
    ]
].copy()
port_events = port_events.rename(columns={
    "ship": "vessel_name",
    "imo": "vessel_id",
    "depPort": "departure_port",
    "arrPort": "arrival_port"
})


In [9]:
port_events

,vessel_name,vessel_id,departure_port,arrival_port,etaSchedule,eta,ata,etdSchedule,etd,atd,arrival_delay_hrs,departure_delay_hrs,port_stay_hrs
0,Megastar,9773064,FIHEL,EETLL,2018-05-04 21:30:00,2018-04-05 21:25:00,2018-04-05 21:23:00,2018-05-04 19:30:00,2018-07-04 15:29:00,2018-04-05 19:18:20,-696.116667,-696.194444,-2.077778
1,Megastar,9773064,FIHEL,EETLL,2018-05-04 21:30:00,2018-04-05 21:25:00,2018-04-05 21:29:00,2018-05-04 19:30:00,2018-07-04 15:29:00,2018-04-05 19:18:20,-696.016667,-696.194444,-2.177778
2,Megastar,9773064,FIHEL,EETLL,2018-05-04 21:30:00,2018-04-05 21:25:00,2018-04-05 21:30:00,2018-05-04 19:30:00,2018-07-04 15:29:00,2018-04-05 19:18:20,-696.000000,-696.194444,-2.194444
3,Star,9364722,EETLL,FIHEL,2018-05-04 21:30:00,2018-04-05 21:26:00,2018-04-05 21:46:00,2018-05-04 19:30:00,2018-07-04 15:25:00,2018-04-05 19:21:17,-695.733333,-696.145278,-2.411944
4,Megastar,9773064,FIHEL,EETLL,2018-05-04 21:30:00,2018-04-05 21:25:00,2018-04-05 21:32:00,2018-05-04 19:30:00,2018-07-04 15:29:00,2018-04-05 19:18:20,-695.966667,-696.194444,-2.227778
...,...,...,...,...,...,...,...,...,...,...,...,...,...
322191,Finlandia,9214379,FIHEL,EETLL,2019-12-03 21:50:00,2019-03-12 21:42:00,2019-03-12 21:47:00,2019-12-03 19:40:00,2019-12-03 19:42:00,2019-03-12 19:42:08,-6384.050000,-6383.964444,-2.081111
322192,Finlandia,9214379,FIHEL,EETLL,2019-12-03 21:50:00,2019-03-12 21:43:00,2019-03-12 21:44:00,2019-12-03 19:40:00,2019-12-03 19:42:00,2019-03-12 19:42:08,-6384.100000,-6383.964444,-2.031111
322193,Finlandia,9214379,FIHEL,EETLL,2019-12-03 21:50:00,2019-03-12 21:43:00,2019-03-12 21:48:00,2019-12-03 19:40:00,2019-12-03 19:42:00,2019-03-12 19:42:08,-6384.033333,-6383.964444,-2.097778
322194,Finlandia,9214379,FIHEL,EETLL,2019-12-03 21:50:00,2019-03-12 21:42:00,2019-03-12 21:38:00,2019-12-03 19:40:00,2019-12-03 19:42:00,2019-03-12 19:42:08,-6384.200000,-6383.964444,-1.931111


## Simple Congestion Indicator

In [10]:
port_events['congestion_flag'] = port_events['port_stay_hrs'].apply(lambda x: 1 if x is not None and x > 24 else 0)

In [11]:
port_events.head()

,vessel_name,vessel_id,departure_port,arrival_port,etaSchedule,eta,ata,etdSchedule,etd,atd,arrival_delay_hrs,departure_delay_hrs,port_stay_hrs,congestion_flag
0,Megastar,9773064,FIHEL,EETLL,2018-05-04 21:30:00,2018-04-05 21:25:00,2018-04-05 21:23:00,2018-05-04 19:30:00,2018-07-04 15:29:00,2018-04-05 19:18:20,-696.116667,-696.194444,-2.077778,0
1,Megastar,9773064,FIHEL,EETLL,2018-05-04 21:30:00,2018-04-05 21:25:00,2018-04-05 21:29:00,2018-05-04 19:30:00,2018-07-04 15:29:00,2018-04-05 19:18:20,-696.016667,-696.194444,-2.177778,0
2,Megastar,9773064,FIHEL,EETLL,2018-05-04 21:30:00,2018-04-05 21:25:00,2018-04-05 21:30:00,2018-05-04 19:30:00,2018-07-04 15:29:00,2018-04-05 19:18:20,-696.000000,-696.194444,-2.194444,0
3,Star,9364722,EETLL,FIHEL,2018-05-04 21:30:00,2018-04-05 21:26:00,2018-04-05 21:46:00,2018-05-04 19:30:00,2018-07-04 15:25:00,2018-04-05 19:21:17,-695.733333,-696.145278,-2.411944,0
4,Megastar,9773064,FIHEL,EETLL,2018-05-04 21:30:00,2018-04-05 21:25:00,2018-04-05 21:32:00,2018-05-04 19:30:00,2018-07-04 15:29:00,2018-04-05 19:18:20,-695.966667,-696.194444,-2.227778,0


In [12]:
port_events.shape

(322196, 14)

In [13]:
PORT_EVENTS_PATH = os.path.join(BASE_PATH, "data", "silver")

port_events.to_csv(
    os.path.join(PORT_EVENTS_PATH, "port_events.csv"),
    index=False
)
